# Momentum Trading Strategy

### Not recommended for use in real-world trades
Purely for the study of momentum trading techniques

# References
[Detail for Monthly Momentum Factor (Mom)](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_mom_factor.html)

[S&P 500](https://www.youtube.com/watch?v=L2nhNvIAyBI)

In [1]:
import yfinance as yf
import pandas as pd
from pandas.tseries.offsets import MonthEnd

In [5]:
# the entire list is way tooo long
# [0] retrieves summary information
# .Symbol retrieves the Symbol column (+index)
tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0].Symbol

In [7]:
# [1] has Added Ticker and Removed Ticker columns
# will apply changes since 2020
df_ = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[1]

In [11]:
df_.Date = pd.to_datetime(df_.Date.Date)
df_ = df_[df_.Date.Date >= "2020-01-01"]

In [29]:
tickers = tickers[~(tickers.isin(df_.Added.Ticker))]
tickers_rem = df_.Removed.Ticker[1:]
tickers = pd.concat([tickers, tickers_rem])
tickers.drop_duplicates(inplace=True)
tickers.dropna(inplace=True)

In [33]:
start = "2020-01-01"
prices, symbols = [], []
for symbol in tickers:
    df = yf.download(symbol, start=start)["Adj Close"]
    if not df.empty:
        #prices = pd.concat([prices, df])
        #symbols = pd.concat([symbols, symbol])
        prices.append(df)
        symbols.append(symbol)

all_prices = pd.concat(prices, axis=1)
all_prices.columns = symbols

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- KSU: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- MXIM: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ALXN: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HFC: No data found, symbol may be delisted
[*********************100%**********************

In [39]:
all_mtl_ret = all_prices.pct_change().resample('M').agg(lambda x : (x + 1).prod() - 1)

In [46]:
all_mtl_ret_12 = all_mtl_ret.rolling(12).agg(lambda x: (x + 1 ).prod() - 1)
all_mtl_ret_12.dropna(inplace=True)

In [55]:
# top ten performers
curr_ = all_mtl_ret_12.iloc[0]
win_ = curr_.nlargest(10)
win_ret = all_mtl_ret.loc[win_.name + MonthEnd(1), win_.index]
win_ret.mean()

-0.006717873181833683

In [60]:
def mom(all_mtl_ret, lookback):
    all_mtl_ret_lb = all_mtl_ret.rolling(lookback).agg(lambda x: (x + 1).prod() - 1)
    all_mtl_ret_lb.dropna(inplace=True)
    
    rets = []
    for row in range(len(all_mtl_ret_lb) - 1):
        curr = all_mtl_ret_lb.iloc[row]
        win = curr.nlargest(50)
        los = curr.nsmallest(50)
        win_ret = all_mtl_ret.loc[win.name + MonthEnd(1), win.index]
        los_ret = all_mtl_ret.loc[los.name + MonthEnd(1), los.index]
        rets.append(win_ret.mean() - los_ret.mean())
        
    return (pd.Series(rets) + 1).prod() - 1

In [61]:
for lookback in range(1, 13):
    print(mom(all_mtl_ret, lookback))

-0.07505469138272292
-0.06444374620824411
-0.27958537527288374
0.1833506964454612
-0.0672429950822303
-0.015475213854133862
-0.016568151897551786
-0.15318628208749185
-0.32607396740346006
-0.341432754215925
-0.1751389451246832
-0.11438769873475763


In [59]:
s_p = yf.download("^GSPC", start=start)
(s_p.Close.pct_change() + 1).prod() - 1

[*********************100%***********************]  1 of 1 completed


0.26779621997767844